In [3]:
import os
import sys
import json
from pathlib import Path
sys.path.append(os.path.abspath('./src'))

import pandas as pd

In [44]:
submission_name = "_tracking_crowd_mot"

submission_name = submission_name + ".csv"

In [51]:
import ast

submission_dir = Path("submissions")
submission_dir.mkdir(exist_ok=True, parents=True)


submission_file = pd.read_csv("submissions/first.csv")
submission_file['objects'] = submission_file['objects'].apply(lambda x: ast.literal_eval(x))

assert isinstance(submission_file['objects'].iloc[0], list), "The first 'objects' entry is not a list!"

In [52]:
df_tracking = pd.read_csv("src/FasterRcnn/outputs/fastercnn_batch_8_epoch_3_mot20_test.csv", header=None)
df_tracking.columns = ["frame", "tracked_id", "x", "y", "w", "h", "confidence", "class"]

df_group = df_tracking.groupby('frame')[["tracked_id", "x", "y", "w", "h", "confidence"]].apply(
    lambda g: g.to_dict(orient="records")
)

In [56]:
mask = submission_file['objective'] == "tracking"
submission_file.loc[mask, 'objects'] = submission_file.loc[mask, 'frame'].map(df_group)


In [57]:
submission_file

,ID,frame,objects,objective
0,0,1.0,"[{'tracked_id': 1, 'x': 948.8416748046876, 'y'...",tracking
1,1,2.0,"[{'tracked_id': 1, 'x': 950.86181640625, 'y': ...",tracking
2,2,3.0,"[{'tracked_id': 1, 'x': 952.3630981445312, 'y'...",tracking
3,3,4.0,"[{'tracked_id': 1, 'x': 954.888671875, 'y': 60...",tracking
4,4,5.0,"[{'tracked_id': 6, 'x': 664.0470581054688, 'y'...",tracking
...,...,...,...,...
5158,5158,-1.0,"{'gt': 'doesn't_exist', 'image': 'test_set/998...",face_reid
5159,5159,-1.0,"{'gt': 'doesn't_exist', 'image': 'test_set/998...",face_reid
5160,5160,-1.0,"{'gt': 'doesn't_exist', 'image': 'test_set/998...",face_reid
5161,5161,-1.0,"{'gt': 'doesn't_exist', 'image': 'test_set/999...",face_reid


In [59]:
submission_file.to_csv(submission_dir / submission_name, index=None)